【問題1】公式チュートリアルモデルを分担して実行  
公式のサンプルモデルを参考に実行をしてみる

In [1]:
# インポート

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
#　データセット取り込み

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
#　バッチ化

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
# Kerasの model subclassing API を使ってtf.kerasモデルを作る

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# モデルのインスタンスを作成
model = MyModel()

In [5]:
# 訓練のためにオプティマイザと損失関数を選ぶ

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [6]:
# モデルの損失と正解率を計測するためのメトリクスを選択

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
# tf.GradientTapeを使ってモデルを訓練

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
# モデルのテスト

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # 次のエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


Epoch 1, Loss: 0.13060563802719116, Accuracy: 96.08000183105469, Test Loss: 0.056669510900974274, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 0.039708979427814484, Accuracy: 98.76000213623047, Test Loss: 0.05488937348127365, Test Accuracy: 98.19999694824219
Epoch 3, Loss: 0.020933110266923904, Accuracy: 99.35333251953125, Test Loss: 0.051970552653074265, Test Accuracy: 98.33999633789062
Epoch 4, Loss: 0.012546710669994354, Accuracy: 99.54499816894531, Test Loss: 0.05135897547006607, Test Accuracy: 98.48999786376953
Epoch 5, Loss: 0.008248469792306423, Accuracy: 99.73333740234375, Test Loss: 0.06322968006134033, Test Accuracy: 98.3699951171875


異なるフレームワークへの書き換え  
「Sprint13 TensorFlow」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更

【問題3】Iris（2値分類）をKerasで学習  
Sequentialモデルを使用

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,))])

In [20]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="/content/sample_data/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X).astype(np.float32)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.float32)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

import tensorflow as tf
model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(4,))])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.1),
              metrics=['accuracy'])

In [23]:
#　実行、accutracyはエポックを１０で９０％を超える結果となった
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val, y_val))

Epoch 1/10
7/7 [==============================] - 0s 19ms/step - loss: 1.8668 - accuracy: 0.5469 - val_loss: 1.0180 - val_accuracy: 0.6250
Epoch 2/10
7/7 [==============================] - 0s 4ms/step - loss: 1.3709 - accuracy: 0.4688 - val_loss: 0.7548 - val_accuracy: 0.3750
Epoch 3/10
7/7 [==============================] - 0s 4ms/step - loss: 0.9950 - accuracy: 0.5312 - val_loss: 1.0567 - val_accuracy: 0.3750
Epoch 4/10
7/7 [==============================] - 0s 4ms/step - loss: 0.7894 - accuracy: 0.4531 - val_loss: 0.6426 - val_accuracy: 0.6250
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6509 - accuracy: 0.6719 - val_loss: 0.7514 - val_accuracy: 0.3750
Epoch 6/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6684 - accuracy: 0.5312 - val_loss: 0.6376 - val_accuracy: 0.4375
Epoch 7/10
7/7 [==============================] - 0s 4ms/step - loss: 0.5928 - accuracy: 0.7344 - val_loss: 0.5706 - val_accuracy: 0.9375
Epoch 8/10
7/7 [=================

【問題4】Iris（多値分類）をKerasで学習  
Sequentialモデルのaddを使用

In [68]:
#　３値でデータを整える
# データセットの読み込み
dataset_path ="/content/sample_data/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X).astype(np.float32)
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.float32)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [69]:
#　OneHotEncoderを実行
y_train_hot=y_train.reshape(-1,)
y_val_hot=y_val.reshape(-1,)
y_test_hot=y_test.reshape(-1,)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train_hot[:, np.newaxis])
y_val_one_hot = enc.transform(y_val_hot[:, np.newaxis])
y_test_one_hot = enc.transform(y_test_hot[:, np.newaxis])

In [98]:
#　今回はaddで書いてみる
import tensorflow as tf
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.Input(shape=(4,)))
model2.add(tf.keras.layers.Dense(100, activation='relu'))
model2.add(tf.keras.layers.Dense(3, activation='softmax'))
model2.output_shape

(None, 3)

In [99]:
model2.compile(loss='categorical_crossentropy',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

In [100]:
# 90%ほどの正解率となった
history = model2.fit(X_train, y_train_one_hot ,
                    batch_size=10,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val, y_val_one_hot))

Epoch 1/10
10/10 [==============================] - 1s 14ms/step - loss: 0.9972 - accuracy: 0.5417 - val_loss: 0.7391 - val_accuracy: 0.6250
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 0.6302 - accuracy: 0.6771 - val_loss: 0.5370 - val_accuracy: 0.7083
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 0.4756 - accuracy: 0.7083 - val_loss: 0.4079 - val_accuracy: 0.7500
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.3651 - accuracy: 0.8125 - val_loss: 0.3748 - val_accuracy: 0.9167
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 0.3089 - accuracy: 0.9271 - val_loss: 0.3412 - val_accuracy: 0.9583
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3055 - accuracy: 0.8958 - val_loss: 0.3446 - val_accuracy: 0.8750
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2962 - accuracy: 0.8646 - val_loss: 0.4345 - val_accuracy: 0.7083
Epoch 8/10
10/10 [=

【問題5】House PricesをKerasで学習  
Functional APIを使用

In [160]:
# PandasのDataFrame型のtrain_dataに格納
import pandas as pd
import numpy as np

train_data=pd.read_csv("/content/sample_data/train.csv")

In [161]:
X =train_data.loc[:,['GrLivArea','YearBuilt']]
y =train_data.loc[:,"SalePrice"]
X_np=np.array(X)
X_np=X_np.astype(float)
y_np=np.array(y)

# データ分割
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np,test_size=0.25,random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

In [162]:
input_data = tf.keras.layers.Input(shape=(2,)) # 入力層
output = tf.keras.layers.Dense(1)(input_data) # 出力層

model3 = tf.keras.Model(inputs=input_data, outputs=output)

In [163]:
#model.summary()
model3.compile(loss='mean_squared_error',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01))
history = model3.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/10
83/83 [==============================] - 0s 2ms/step - loss: 37710651392.0000 - val_loss: 38593613824.0000
Epoch 2/10
83/83 [==============================] - 0s 1ms/step - loss: 36669378560.0000 - val_loss: 37551689728.0000
Epoch 3/10
83/83 [==============================] - 0s 1ms/step - loss: 35652800512.0000 - val_loss: 36529901568.0000
Epoch 4/10
83/83 [==============================] - 0s 1ms/step - loss: 34655977472.0000 - val_loss: 35535134720.0000
Epoch 5/10
83/83 [==============================] - 0s 1ms/step - loss: 33683372032.0000 - val_loss: 34562772992.0000
Epoch 6/10
83/83 [==============================] - 0s 1ms/step - loss: 32732971008.0000 - val_loss: 33605400576.0000
Epoch 7/10
83/83 [==============================] - 0s 1ms/step - loss: 31799418880.0000 - val_loss: 32672851968.0000
Epoch 8/10
83/83 [==============================] - 0s 2ms/step - loss: 30895900672.0000 - val_loss: 31767728128.0000
Epoch 9/10
83/83 [==============================] - 0s 2

【問題6】MNISTをKerasで学習

In [152]:
# データの取り込みと前処理

#　データの取り込みと処理を行う
import numpy as np
import math 
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [153]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [154]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [155]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_hot, y_val_hot, = train_test_split(X_train, y_train_one_hot, test_size=0.20)

In [158]:
input_data = tf.keras.layers.Input(shape=(784,)) # 入力層

x = tf.keras.layers.Dense(50, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(100, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(x)

model4 = tf.keras.Model(inputs=input_data, outputs=output)

In [159]:
model4.compile(loss='categorical_crossentropy',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model4.fit(X_train, y_train_hot,
                    batch_size=10,
                    epochs=10,
                    verbose=1)

Epoch 1/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.3795 - accuracy: 0.8959
Epoch 2/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2797 - accuracy: 0.9272
Epoch 3/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2553 - accuracy: 0.9376
Epoch 4/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2323 - accuracy: 0.9440
Epoch 5/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2247 - accuracy: 0.9464
Epoch 6/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2240 - accuracy: 0.9479
Epoch 7/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2169 - accuracy: 0.9508
Epoch 8/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2089 - accuracy: 0.9536
Epoch 9/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.2104 - accuracy: 0.9535
Epoch 10/10
4800/4800 [==============================] - 6s 1ms/step - loss: 0.194

MNISTでも95%ほどの正解率となり、しっかりと動いていそうである。